# 0. Imports & Data Load

In [24]:
import pandas as pd
from igraph import Graph
import disease_process_proteins
import metrics_functions
import importlib
import numpy as np
from ast import literal_eval
import pickle
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import plotly.graph_objects as go
from plotly.offline import iplot

In [25]:
reactome_proteins_indexes_df = pd.read_csv("../../data/processed/reactome_proteins_indexes.csv", sep=',', header=0)
disgenet_proteins_indexes_df = pd.read_csv("../../data/processed/disgenet_proteins_indexes.csv", sep=',', header=0)
protein_ids_df = pd.read_csv("../../data/processed/protein_ids.csv", sep=',', header=0)
adj_matrix = np.load("../../data/processed/adjacency_matrix.npy")
ppi_80 = np.load("../../data/processed/ppis/ppis_red80.npy", allow_pickle=True)

In [26]:
reactome_proteins_indexes_df['protein_index'] = reactome_proteins_indexes_df['protein_index'].apply(literal_eval)
disgenet_proteins_indexes_df['protein_index'] = disgenet_proteins_indexes_df['protein_index'].apply(literal_eval)
reactome_proteins_indexes_df['proteins_ids'] = reactome_proteins_indexes_df['proteins_ids'].apply(literal_eval)
disgenet_proteins_indexes_df['proteins_ids'] = disgenet_proteins_indexes_df['proteins_ids'].apply(literal_eval)

In [27]:
graph = Graph.Read_GML("../../data/processed/graph")

# 1. Target Process Metrics

## 1.1. Hypergeometric Test

For a given protein and given process, the number of interactions formed between the protein and the proteins of the process are summed. The hypergeometric test is calculated given this number, the number total number of interactions of the protein,the number of proteins of the process and the total number of proteins. For an easier analysis, the p-values are presented in a -log10 form (higher -log10 means lower p-value).

In [6]:
importlib.reload(metrics_functions)
proteins_neglogp_byprocess = metrics_functions.hypergeometric_test(graph, reactome_proteins_indexes_df, adj_matrix)

  0%|          | 0/21302 [00:00<?, ?it/s]

In [7]:
proteins_neglogp_byprocess_df = pd.DataFrame.from_dict(proteins_neglogp_byprocess, orient='index')
proteins_neglogp_byprocess_df.to_csv('../../data/processed/metrics/neglogpvalue_reactome.csv')
proteins_neglogp_byprocess_df.head()

,R-HSA-1031716,R-HSA-112379,R-HSA-112385,R-HSA-1168640,R-HSA-1214188,R-HSA-1234159,R-HSA-141409,R-HSA-141422,R-HSA-141431,R-HSA-141439,...,R-HSA-9670149,R-HSA-9698928,R-HSA-9710490,R-HSA-977224,R-HSA-983140,R-HSA-983147,R-HSA-983156,R-HSA-983157,R-HSA-983259,R-HSA-983266
7SK.1,0.006866,0.005453,0.005453,0.004794,0.005547,0.005830,0.009034,0.009034,0.009034,0.009034,...,0.006112,0.005736,0.006207,0.005359,0.023898,0.020290,0.023898,0.023898,0.005547,0.005547
A1BG,0.116811,0.092776,0.092776,2.585937,0.094378,2.407728,0.153698,0.153698,0.153698,0.153698,...,0.103990,0.097582,0.105593,0.091174,4.980567,5.414976,4.980567,4.980567,0.094378,0.094378
A1CF,0.116811,0.092776,0.092776,2.585937,0.094378,0.099184,0.153698,0.153698,0.153698,0.153698,...,0.103990,0.097582,0.105593,0.091174,0.406565,0.345189,0.406565,0.406565,0.094378,0.094378
A2M,1.062935,1.155290,1.155290,1.219699,4.214351,0.571015,0.884863,0.884863,0.884863,0.884863,...,2.326305,1.132384,3.936025,4.301489,2.914667,3.391239,2.914667,2.914667,1.147370,1.147370
A2ML1,1.851789,0.150153,0.150153,2.154418,0.152746,0.160524,0.248752,0.248752,0.248752,0.248752,...,0.168303,0.157931,0.170896,0.147561,0.658006,0.558671,0.658006,0.658006,0.152746,0.152746


In [ ]:
importlib.reload(metrics_functions)
proteins_neglogp_bydisease = metrics_functions.hypergeometric_test(graph, disgenet_proteins_indexes_df, adj_matrix)

In [ ]:
proteins_neglogp_bydisease_df = pd.DataFrame.from_dict(proteins_neglogp_bydisease, orient='index')
proteins_neglogp_bydisease_df.to_csv('../../data/processed/metrics/neglogpvalue_disgenet.csv')
proteins_neglogp_bydisease_df.head()

## 1.2. Shortest Paths

In [14]:
importlib.reload(metrics_functions)
process_shortest_paths = metrics_functions.shortest_paths(graph, reactome_proteins_indexes_df)

  0%|          | 0/3795 [00:00<?, ?it/s]

In [15]:
process_shortest_paths_df = pd.DataFrame.from_dict(process_shortest_paths)
process_shortest_paths_df = process_shortest_paths_df.rename(index=dict(zip(list(process_shortest_paths_df.index),list(protein_ids_df['protein_id'].values))))
process_shortest_paths_df.to_csv('../../data/processed/metrics/process_shortest_paths.csv')
print(process_shortest_paths_df.shape)
process_shortest_paths_df.head()

(21302, 3795)


,CTSS,RXFP1,KRTAP1-4,COMMD5,FERMT2,PLET1,COPZ2,OFD1,IQCB1,GBP1,...,SKA1,RPL41,TBP,FBXL19,GRIN1,NPY5R,HP,PKM,UBTF,DCUN1D3
7SK.1,3,4,4,3,3,3,3,3,3,3,...,3,3,2,2,2,3,3,3,3,3
A1BG,3,3,2,3,3,3,2,2,2,3,...,3,3,2,2,2,2,2,2,2,3
A1CF,3,3,3,2,3,3,3,2,3,3,...,2,3,2,3,3,3,2,2,3,2
A2M,2,3,3,2,2,3,3,2,2,2,...,2,2,2,3,2,2,2,2,2,2
A2ML1,2,2,3,2,2,3,3,2,3,3,...,2,3,2,2,2,3,2,2,2,3


## 1.3. Closeness

In [6]:
process_shortest_paths_df = pd.read_csv("../../data/processed/metrics/process_shortest_paths.csv", sep=',', header=0, index_col=0)

In [30]:
importlib.reload(metrics_functions)
closeness = metrics_functions.closeness(process_shortest_paths_df, reactome_proteins_indexes_df)

  0%|          | 0/433 [00:00<?, ?it/s]

In [31]:
closeness_df = pd.DataFrame.from_dict(closeness)
closeness_df = closeness_df.rename(index=dict(zip(list(closeness_df.index),list(protein_ids_df['protein_id'].values))))
closeness_df.to_csv('../../data/processed/metrics/process_closeness.csv')
print(closeness_df.shape)
closeness_df.head()

(21302, 433)


,R-HSA-1031716,R-HSA-112379,R-HSA-112385,R-HSA-1168640,R-HSA-1214188,R-HSA-1234159,R-HSA-141409,R-HSA-141422,R-HSA-141431,R-HSA-141439,...,R-HSA-9670149,R-HSA-9698928,R-HSA-9710490,R-HSA-977224,R-HSA-983140,R-HSA-983147,R-HSA-983156,R-HSA-983157,R-HSA-983259,R-HSA-983266
7SK.1,0.324444,0.339181,0.339181,0.335526,0.310526,0.342541,0.336842,0.336842,0.336842,0.336842,...,0.325000,0.309645,0.314286,0.309783,0.329427,0.327744,0.329427,0.329427,0.327778,0.327778
A1BG,0.386243,0.386667,0.386667,0.490385,0.355422,0.484375,0.380952,0.380952,0.380952,0.380952,...,0.365169,0.350575,0.364641,0.358491,0.415435,0.420744,0.415435,0.415435,0.398649,0.398649
A1CF,0.412429,0.371795,0.371795,0.414634,0.388158,0.405229,0.391837,0.391837,0.391837,0.391837,...,0.396341,0.391026,0.392857,0.390411,0.383915,0.383244,0.383915,0.383915,0.390728,0.390728
A2M,0.477124,0.464000,0.464000,0.476636,0.433824,0.476923,0.442396,0.442396,0.442396,0.442396,...,0.442177,0.423611,0.440000,0.431818,0.463370,0.464363,0.463370,0.463370,0.472000,0.472000
A2ML1,0.453416,0.432836,0.432836,0.425000,0.418440,0.421769,0.444444,0.444444,0.444444,0.444444,...,0.433333,0.420690,0.428571,0.422222,0.428088,0.430862,0.428088,0.428088,0.450382,0.450382


## 1.4. Betweenness

In [18]:
process_shortest_paths_df = pd.read_csv('../../data/processed/metrics/process_shortest_paths.csv', index_col=0)

In [19]:
importlib.reload(metrics_functions)
process_betweenness = metrics_functions.betweenness(process_shortest_paths_df, reactome_proteins_indexes_df, graph)

  0%|          | 0/433 [00:00<?, ?it/s]

In [20]:
process_betweenness_df = pd.DataFrame.from_dict(process_betweenness)
process_betweenness_df = process_betweenness_df.rename(index=dict(zip(list(process_betweenness_df.index),list(protein_ids_df['protein_id'].values))))
process_betweenness_df.fillna(value=0, inplace=True)
process_betweenness_df.to_csv('../../data/processed/metrics/process_betweenness.csv')
print(process_betweenness_df.shape)
process_betweenness_df.head()

(21302, 433)


,R-HSA-1031716,R-HSA-112379,R-HSA-112385,R-HSA-1168640,R-HSA-1214188,R-HSA-1234159,R-HSA-141409,R-HSA-141422,R-HSA-141431,R-HSA-141439,...,R-HSA-9670149,R-HSA-9698928,R-HSA-9710490,R-HSA-977224,R-HSA-983140,R-HSA-983147,R-HSA-983156,R-HSA-983157,R-HSA-983259,R-HSA-983266
7SK.1,0.013318,0.0,0.0,0.002353,0.013442,0.000529,0.005921,0.005921,0.005921,0.005921,...,0.018750,0.014208,0.012587,0.013784,0.006431,0.006216,0.006431,0.006431,0.042665,0.042665
A1BG,0.013318,0.0,0.0,0.002353,0.013442,0.000529,0.007018,0.007018,0.007018,0.007018,...,0.017308,0.013661,0.012587,0.013784,0.004611,0.003912,0.004611,0.004611,0.038574,0.038574
A1CF,0.028158,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.004172,0.002173,0.004172,0.004172,0.000000,0.000000
A2M,0.029300,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A2ML1,0.002664,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000063,0.000043,0.000063,0.000063,0.000000,0.000000


## 1.5. Random-walks with restart

In [21]:
importlib.reload(metrics_functions)
process_rwr = metrics_functions.random_walk_restart(graph, reactome_proteins_indexes_df)

  0%|          | 0/433 [00:00<?, ?it/s]

In [22]:
process_rwr_df = pd.DataFrame.from_dict(process_rwr)
process_rwr_df = process_rwr_df.rename(index=dict(zip(list(process_rwr_df.index),list(protein_ids_df['protein_id'].values))))
process_rwr_df.to_csv('../../data/processed/metrics/process_rwr.csv')
print(process_rwr_df.shape)
process_rwr_df.head()

(21302, 433)


,R-HSA-1031716,R-HSA-112379,R-HSA-112385,R-HSA-1168640,R-HSA-1214188,R-HSA-1234159,R-HSA-141409,R-HSA-141422,R-HSA-141431,R-HSA-141439,...,R-HSA-9670149,R-HSA-9698928,R-HSA-9710490,R-HSA-977224,R-HSA-983140,R-HSA-983147,R-HSA-983156,R-HSA-983157,R-HSA-983259,R-HSA-983266
7SK.1,0.000001,9.798433e-07,9.798433e-07,7.379129e-07,0.000004,8.470797e-07,0.000001,0.000001,0.000001,0.000001,...,0.000003,0.000003,0.000003,0.000004,8.603440e-07,8.440027e-07,8.603440e-07,8.603440e-07,7.677677e-07,7.677677e-07
A1BG,0.000015,1.450999e-05,1.450999e-05,4.661402e-05,0.000015,4.201305e-05,0.000015,0.000015,0.000015,0.000015,...,0.000014,0.000014,0.000015,0.000015,3.276858e-05,3.538742e-05,3.276858e-05,3.276858e-05,1.716237e-05,1.716237e-05
A1CF,0.000014,1.351225e-05,1.351225e-05,2.972250e-05,0.000014,2.119540e-05,0.000014,0.000014,0.000014,0.000014,...,0.000013,0.000014,0.000014,0.000014,1.480853e-05,1.497975e-05,1.480853e-05,1.480853e-05,1.561255e-05,1.561255e-05
A2M,0.000090,9.780978e-05,9.780978e-05,8.531566e-05,0.000118,8.424280e-05,0.000082,0.000082,0.000082,0.000082,...,0.000109,0.000087,0.000115,0.000119,1.119695e-04,1.162859e-04,1.119695e-04,1.119695e-04,1.180395e-04,1.180395e-04
A2ML1,0.000034,2.198489e-05,2.198489e-05,2.345548e-05,0.000029,2.060198e-05,0.000023,0.000023,0.000023,0.000023,...,0.000025,0.000030,0.000029,0.000030,2.305171e-05,2.329654e-05,2.305171e-05,2.305171e-05,2.567130e-05,2.567130e-05


# 2. Metrics with Reduced (80%) Network

In [8]:
importlib.reload(metrics_functions)
ppi80_hyper, ppi80_closeness, ppi80_betweenness, ppi80_rwr = metrics_functions.multiple_metrics(ppi_80, reactome_proteins_indexes_df)

  0%|          | 0/10 [00:00<?, ?it/s]

Generating Graph
Shortest Paths Computation:


  0%|          | 0/3789 [00:00<?, ?it/s]

Closeness Computation:


  0%|          | 0/433 [00:00<?, ?it/s]

...

  0%|          | 0/433 [00:00<?, ?it/s]

Betweenness Computation:


  0%|          | 0/433 [00:00<?, ?it/s]

Random-Walks with Restart Computation:


  0%|          | 0/433 [00:00<?, ?it/s]

In [9]:
ppi80_hyper.to_csv('../../data/processed/metrics/process_ppi80_hyper.csv')
ppi80_closeness.to_csv('../../data/processed/metrics/process_ppi80_closeness.csv')
ppi80_betweenness.to_csv('../../data/processed/metrics/process_ppi80_betweenness.csv')
ppi80_rwr.to_csv('../../data/processed/metrics/process_ppi80_rwr.csv')

# 3. NMAD Computation

In [35]:
proteins_neglogp_byprocess_df = pd.read_csv('../../data/processed/metrics/neglogpvalue_reactome.csv', index_col=0)
process_closeness_df = pd.read_csv('../../data/processed/metrics/process_closeness.csv', index_col=0)
process_betweenness_df = pd.read_csv('../../data/processed/metrics/process_betweenness.csv', index_col=0)
process_rwr_df = pd.read_csv('../../data/processed/metrics/process_rwr.csv', index_col=0)

In [36]:
cols = list(process_betweenness_df.columns)[1:]
process_betweenness_df[cols] = process_betweenness_df[cols].replace({'0':np.nan, 0:np.nan})

In [37]:
def NMAD(x):
    med = np.nanmedian(x)
    diff = abs(x-med)
    NMAD = np.nanmedian(diff)/med
    return NMAD

In [41]:
reactome_proteins_indexes_df.set_index(['process'], inplace=True)

In [42]:
importlib.reload(disease_process_proteins)
hyper_process_proteins, hyper_remain_proteins = disease_process_proteins.process_proteins_selection(proteins_neglogp_byprocess_df, reactome_proteins_indexes_df, multiple_df=False)
closeness_process_proteins, closeness_remain_proteins = disease_process_proteins.process_proteins_selection(process_closeness_df, reactome_proteins_indexes_df, multiple_df=False)
betweenness_process_proteins, betweenness_remain_proteins = disease_process_proteins.process_proteins_selection(process_betweenness_df, reactome_proteins_indexes_df, multiple_df=False)
rwr_process_proteins, rwr_remain_proteins = disease_process_proteins.process_proteins_selection(process_rwr_df, reactome_proteins_indexes_df, multiple_df=False)

  0%|          | 0/433 [00:00<?, ?it/s]

  0%|          | 0/433 [00:00<?, ?it/s]

  0%|          | 0/433 [00:00<?, ?it/s]

  0%|          | 0/433 [00:00<?, ?it/s]

In [43]:
hyper_results = proteins_neglogp_byprocess_df.apply(NMAD)
closeness_results = process_closeness_df.apply(NMAD)
betweenness_results = process_betweenness_df.apply(NMAD)
rwr_results = process_rwr_df.apply(NMAD)

<ipython-input-37-8d93b6046676>:4: RuntimeWarning:

invalid value encountered in double_scalars



In [44]:
hyper_results_process_proteins = hyper_process_proteins.apply(NMAD)
hyper_results_remain_proteins = hyper_remain_proteins.apply(NMAD)
closeness_results_process_proteins = closeness_process_proteins.apply(NMAD)
closeness_results_remain_proteins = closeness_remain_proteins.apply(NMAD)
betweenness_results_process_proteins = betweenness_process_proteins.apply(NMAD)
betweenness_results_remain_proteins = betweenness_remain_proteins.apply(NMAD)
rwr_results_process_proteins = rwr_process_proteins.apply(NMAD)
rwr_results_remain_proteins = rwr_remain_proteins.apply(NMAD)

<ipython-input-37-8d93b6046676>:4: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\alexf\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1113: RuntimeWarning:

All-NaN slice encountered



In [45]:
hyper_results.to_csv('../../data/processed/nmad/hyper_process_all.csv', header=['NMAD'], index_label = ['process'])
closeness_results.to_csv('../../data/processed/nmad/closeness_process_all.csv', header=['NMAD'], index_label = ['process'])
betweenness_results.to_csv('../../data/processed/nmad/betweenness_process_all.csv', header=['NMAD'], index_label = ['process'])
rwr_results.to_csv('../../data/processed/nmad/rwr_process_all.csv', header=['NMAD'], index_label = ['process'])

hyper_results_process_proteins.to_csv('../../data/processed/nmad/hyper_process_proteins.csv', header=['NMAD'], index_label = ['process'])
hyper_results_remain_proteins.to_csv('../../data/processed/nmad/hyper_process_remain_proteins.csv', header=['NMAD'], index_label = ['process'])
closeness_results_process_proteins.to_csv('../../data/processed/nmad/closeness_process_proteins.csv', header=['NMAD'], index_label = ['process'])
closeness_results_remain_proteins.to_csv('../../data/processed/nmad/closeness_process_remain_proteins.csv', header=['NMAD'], index_label = ['process'])
betweenness_results_process_proteins.to_csv('../../data/processed/nmad/betweenness_process_proteins.csv', header=['NMAD'], index_label = ['process'])
betweenness_results_remain_proteins.to_csv('../../data/processed/nmad/betweenness_process_remain_proteins.csv', header=['NMAD'], index_label = ['process'])
rwr_results_process_proteins.to_csv('../../data/processed/nmad/rwr_process_proteins.csv', header=['NMAD'], index_label = ['process'])
rwr_results_remain_proteins.to_csv('../../data/processed/nmad/rwr_process_remain_proteins.csv', header=['NMAD'], index_label = ['process'])

In [46]:
ppi80_hyper = pd.read_csv("../../data/processed/metrics/process_ppi80_hyper.csv", sep=',', header=0, index_col=0)
ppi80_closeness = pd.read_csv("../../data/processed/metrics/process_ppi80_closeness.csv", sep=',', header=0, index_col=0)
ppi80_betweenness = pd.read_csv("../../data/processed/metrics/process_ppi80_betweenness.csv", sep=',', header=0, index_col=0)
ppi80_rwr = pd.read_csv("../../data/processed/metrics/process_ppi80_rwr.csv", sep=',', header=0, index_col=0)

In [47]:
cols = list(ppi80_betweenness.columns)[1:]
ppi80_betweenness[cols] = ppi80_betweenness[cols].replace({'0':np.nan, 0:np.nan})

In [48]:
importlib.reload(disease_process_proteins)
ppi80_hyper_process_proteins, ppi80_hyper_remain_proteins = disease_process_proteins.process_proteins_selection(ppi80_hyper, reactome_proteins_indexes_df)
ppi80_closeness_process_proteins, ppi80_closeness_remain_proteins = disease_process_proteins.process_proteins_selection(ppi80_closeness, reactome_proteins_indexes_df)
ppi80_betweenness_process_proteins, ppi80_betweenness_remain_proteins = disease_process_proteins.process_proteins_selection(ppi80_betweenness, reactome_proteins_indexes_df)
ppi80_rwr_process_proteins, ppi80_rwr_remain_proteins = disease_process_proteins.process_proteins_selection(ppi80_rwr, reactome_proteins_indexes_df)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [49]:
hyper_results_80 = ppi80_hyper.groupby(level=0).transform(NMAD).mean(axis=0)
closeness_results_80 = ppi80_closeness.groupby(level=0).transform(NMAD).mean(axis=0)
betweenness_results_80 = ppi80_betweenness.groupby(level=0).transform(NMAD).mean(axis=0)
rwr_results_80 = ppi80_rwr.groupby(level=0).transform(NMAD).mean(axis=0)

In [50]:
hyper_results_process_proteins_80 = ppi80_hyper_process_proteins.groupby(level=0).transform(NMAD).mean(axis=0)
hyper_results_remain_proteins_80 = ppi80_hyper_remain_proteins.groupby(level=0).transform(NMAD).mean(axis=0)
closeness_results_process_proteins_80 = ppi80_closeness_process_proteins.groupby(level=0).transform(NMAD).mean(axis=0)
closeness_results_remain_proteins_80 = ppi80_closeness_remain_proteins.groupby(level=0).transform(NMAD).mean(axis=0)
betweenness_results_process_proteins_80 = ppi80_betweenness_process_proteins.groupby(level=0).transform(NMAD).mean(axis=0)
betweenness_results_remain_proteins_80 = ppi80_betweenness_remain_proteins.groupby(level=0).transform(NMAD).mean(axis=0)
rwr_results_process_proteins_80 = ppi80_rwr_process_proteins.groupby(level=0).transform(NMAD).mean(axis=0)
rwr_results_remain_proteins_80 = ppi80_rwr_remain_proteins.groupby(level=0).transform(NMAD).mean(axis=0)

In [51]:
hyper_results_80.to_csv('../../data/processed/nmad/hyper_80_process_all.csv', header=['NMAD'], index_label = ['process'])
closeness_results_80.to_csv('../../data/processed/nmad/closeness_80_process_all.csv', header=['NMAD'], index_label = ['process'])
betweenness_results_80.to_csv('../../data/processed/nmad/betweenness_80_process_all.csv', header=['NMAD'], index_label = ['process'])
rwr_results_80.to_csv('../../data/processed/nmad/rwr_80_process_all.csv', header=['NMAD'], index_label = ['process'])

hyper_results_process_proteins_80.to_csv('../../data/processed/nmad/hyper_80_process_proteins.csv', header=['NMAD'], index_label = ['process'])
hyper_results_remain_proteins_80.to_csv('../../data/processed/nmad/hyper_80_process_remain_proteins.csv', header=['NMAD'], index_label = ['process'])
closeness_results_process_proteins_80.to_csv('../../data/processed/nmad/closeness_80_process_proteins.csv', header=['NMAD'], index_label = ['process'])
closeness_results_remain_proteins_80.to_csv('../../data/processed/nmad/closeness_80_process_remain_proteins.csv', header=['NMAD'], index_label = ['process'])
betweenness_results_process_proteins_80.to_csv('../../data/processed/nmad/betweenness_80_process_proteins.csv', header=['NMAD'], index_label = ['process'])
betweenness_results_remain_proteins_80.to_csv('../../data/processed/nmad/betweenness_80_process_remain_proteins.csv', header=['NMAD'], index_label = ['process'])
rwr_results_process_proteins_80.to_csv('../../data/processed/nmad/rwr_80_process_proteins.csv', header=['NMAD'], index_label = ['process'])
rwr_results_remain_proteins_80.to_csv('../../data/processed/nmad/rwr_80_process_remain_proteins.csv', header=['NMAD'], index_label = ['process'])